<img src="logos/‎cover‎001.png" alt="Drawing"/>

In [3]:
# imports
import pyxdf
import pandas as pd
import copy
import datetime
import itertools
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
import seaborn as sns
from tqdm.notebook import tqdm

# Load data
Test files with different durations are stored in the folder "data". Before loading them, we want to access their path and save them in a dictionary.

### Task 1:
- Get all files from "data" folder, sort them alphabetically and save them to a variable "files"
Hint: os.listdir()

In [20]:
# get all files from the folder "data"
files = os.listdir("data")
# sort them alphabetically
files.sort()
files

['lsl_test1.xdf', 'lsl_test2.xdf', 'lsl_test3.xdf']

In [21]:
# save all files to an empty dictionary "recordings"
recordings = {}

for i, file in enumerate(files):  # store and display all files
    created = os.path.getmtime(f"data/{file}")  # creation timestamp
    created = datetime.datetime.fromtimestamp(created)  # translate as datetime
    created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
    recordings[i] = {"file": file, "created": created}

files = [f.split(".")[0] for f in files]
# display the recordings and metadata
display(recordings)

{0: {'file': 'lsl_test1.xdf', 'created': '05.08.2022 16:27'},
 1: {'file': 'lsl_test2.xdf', 'created': '05.08.2022 17:09'},
 2: {'file': 'lsl_test3.xdf', 'created': '05.08.2022 17:27'}}

# Extensible Data Format (XDF)

.XDF is a format to store multiple channels of time series data with specific meta information.
For the latency test, we stored 3 streams
- Unity: 'Visual',
- EEG: 'openvibeMarkers', 'openvibeSignal'

The loaded .XDF file results in a list of dictionaries with two main components:

1. **'time_series':** the information sent from Unity and EEG to via LSL
2. **'time_stamps':** the time stamps for each datapoint received

### Task 2:
Load data streams for test 1
Hint: look at [.XDF website info](https://pypi.org/project/pyxdf/)

In [49]:
# load data
streams, _ = pyxdf.load_xdf(f"data/{recordings[0]['file']}")
display(streams)

[{'info': defaultdict(list,
              {'name': ['openvibeMarkers'],
               'type': ['Markers'],
               'channel_count': ['1'],
               'channel_format': ['int32'],
               'source_id': ['(0xb4e83c55, 0xfbc1c4eb)'],
               'nominal_srate': ['0.000000000000000'],
               'version': ['1.100000000000000'],
               'created_at': ['237832.7103915000'],
               'uid': ['8cc5f0ca-0733-4052-9e83-c41e7c935a67'],
               'session_id': ['default'],
               'hostname': ['wd-left'],
               'v4address': [None],
               'v4data_port': ['16573'],
               'v4service_port': ['16573'],
               'v6address': [None],
               'v6data_port': ['16573'],
               'v6service_port': ['16573'],
               'desc': [defaultdict(list,
                            {'channels': [defaultdict(list,
                                          {'channel': [defaultdict(list,
                                

### Accessing streams
Once we load the streams, we can access each list in the dictionary by indicating for example the component we want to access such as stream (an index), 'info', time_stamps or time_series, etc.

For specific information:
- streams[2]['info']['name']

For a particular stream component:
- streams[2]['time_stamps']

For a specific eeg channel
- streams[2]["time_series"][:,64]

In [162]:
# access the second stream's time_stamps
streams[2]['time_stamps']

array([241076.51462992, 241076.51560648, 241076.51658304, ...,
       241805.01170612, 241805.01268268, 241805.01365924])

In [159]:
# print all stream names and index in stream list
s_names = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
s_names

{'openvibeMarkers': 0, 'Visual': 1, 'openvibeSignal': 2}

### Task 3:
Define a function that given the .xdf streams, it returns the relevant streams _Visual_ and _openvibeSignal_

In [68]:
# define a function to select Visual and openvibeSignal from streams
def select_streams(streams):
    # stream names
    u_ch_name = "Visual"
    e_ch_name = "openvibeSignal"

    # get all current streams with their positions on the recording
    # example: {'Diode': 0, 'Audio': 1, 'openvibeSignal': 2}
    s_names = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}

    # store and return their positions
    u = s_names[u_ch_name]  # unity stream channel
    e = s_names[e_ch_name]  # eeg stream channel (photodiode)
    return u, e

In [164]:
# extract streams info for all files
overview_df = pd.DataFrame()

for r in recordings:
    # current filename
    file = recordings[r]["file"]
    # load data
    streams, _ = pyxdf.load_xdf(f"data/{file}")
    # select stream channels
    u_ch, e_ch = select_streams(streams)
    # computer host name
    u_host = streams[u_ch]["info"]["hostname"]
    e_host = streams[e_ch]["info"]["hostname"]

    # select timestamps for each stream
    u_ts = streams[u_ch]["time_stamps"]
    e_ts = streams[e_ch]["time_stamps"]

    # Data from streams
    u_data = streams[u_ch]["time_series"][:,1]
    # eeg only channel 64 where the diode was connected
    e_data = streams[e_ch]["time_series"][:,64]

    # calculate Unity - eeg  time difference at start of recording
    # in milliseconds
    diff_ts = (u_ts[0] - e_ts[0]) * 1000

    # calculate recoding duration
    unity_duration =  (u_ts[-1] - u_ts[0]) / 60
    eeg_duration =  (e_ts[-1] - e_ts[0]) / 60

    # calculate sampling rate (FPS)
    unity_sr = 1 / (unity_duration * 60 / len(u_ts))
    eeg_sr = 1 / (eeg_duration * 60 / len(e_ts))

    overview_df = overview_df.append({"file_name":file,
                          "u_duration":unity_duration,
                          "eeg_duration":eeg_duration,
                          "u_fps":unity_sr,
                          "eeg_fps":eeg_sr,
                          "u_host":u_host,"eeg_host":e_host},ignore_index=True)

In [165]:
overview_df.T

,0,1,2
file_name,lsl_test1.xdf,lsl_test2.xdf,lsl_test3.xdf
u_duration,3.916573,7.316522,12.141584
eeg_duration,3.91665,7.31665,12.14165
u_fps,90.006411,90.001775,90.001986
eeg_fps,1024.00425,1024.002292,1024.001364
u_host,[ml03],[ml03],[ml03]
eeg_host,[wd-left],[wd-left],[wd-left]
